In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import truncnorm
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points

Read dwelling types

In [2]:
dwellingTypes = pd.read_csv('./Data/DwellingTypesByZone2006.csv')

Read Zonal Profile data

In [3]:
profileData = pd.read_csv('./Data/GTAProfileData.csv')

Create ZoneSystem file which contains the zones to be used in the sim

In [4]:
zoneSystem = pd.DataFrame(data=profileData['DAUID'], columns=['DAUID'], dtype=str)
zoneSystem = zoneSystem[zoneSystem['DAUID'].str.len()==8]

Read Shapefile Zones

In [5]:
zoneShape:gpd.GeoDataFrame = gpd.read_file('./Data/TorontoZoneShape/TorontoZones_26917.shp')
zoneShape = zoneShape[zoneShape['DAUID'].astype(str).isin(zoneSystem['DAUID'])]

Create Zone Dataframe that holds all data

In [6]:
zoneData = pd.DataFrame(columns=['DAUID', 'Population', 'Employment rate', 'Area', "Total - Structural type of dwelling"])

In [7]:
for i, zone in zoneSystem.iterrows():
    print(zone['DAUID'])
    numDwellings=200.0
    try:numDwellings = dwellingTypes.loc[dwellingTypes['Geography'].astype(str)==zone['DAUID'], 'Total - Structural type of dwelling'].iloc[0]
    except:pass

    zoneData.loc[len(zoneData)] = [zone['DAUID'], profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Population"].iloc[0], profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Employment rate"].iloc[0], float(zoneShape.loc[zoneShape['DAUID'].astype(str)==zone['DAUID'], 'geometry'].iloc[0].area), numDwellings]

35190002
35190003
35190004
35190005
35190006
35190009
35190011
35190012
35190013
35190015
35190016
35190017
35190018
35190019
35190020
35190021
35190022
35190026
35190027
35190028
35190029
35190030
35190031
35190032
35190033
35190034
35190035
35190036
35190037
35190041
35190042
35190043
35190044
35190045
35190046
35190047
35190048
35190049
35190050
35190051
35190052
35190053
35190054
35190055
35190056
35190057
35190058
35190059
35190061
35190062
35190065
35190066
35190067
35190068
35190069
35190070
35190071
35190072
35190073
35190074
35190075
35190076
35190077
35190078
35190079
35190080
35190081
35190082
35190083
35190084
35190085
35190086
35190087
35190088
35190089
35190090
35190091
35190092
35190093
35190094
35190095
35190096
35190097
35190098
35190099
35190100
35190101
35190102
35190103
35190104
35190105
35190106
35190107
35190108
35190109
35190110
35190111
35190112
35190113
35190114
35190115
35190117
35190118
35190119
35190120
35190121
35190122
35190123
35190124
35190125
35190126
3

In [8]:
zoneData.to_csv('./Data/combinedZoneData.csv', index=False)

<span style="font-size:5em;">CREATE PERSONS</span>

In [9]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [10]:
salary_distribution_employed = get_truncated_normal(mean=70000, sd=40000, low=30000, upp=200000)
salary_distribution_retired = get_truncated_normal(mean=40000, sd=20000, low=15000, upp=70000)

In [11]:
totalPersons = zoneData['Population'].astype(float).sum()

In [12]:
employment_incomes = salary_distribution_employed.rvs(int(totalPersons*.9))
retired_incomes = salary_distribution_retired.rvs(int(totalPersons*.1))